# CUDA utilities

**Cuda availability**

Chek if we have a GPU that is compatible with CUDA.
In a module one can add the following snippet (`__init__` will then be automatically called)
```julia
const _cuda = Ref(false)
function __init__()
    _cuda[] = CUDA.functional()
end;
```

> Reference: https://cuda.juliagpu.org/stable/installation/conditional/

In [1]:
#nbx --fname=src/MyCudaUtils.jl --jl_module=MyCudaUtils
using CUDA
using BenchmarkTools

const _cuda = Ref(false)
function __init__()
    _cuda[] = CUDA.functional()
end;
DeviceArray(x) = _cuda[] ? CuArray(x) : Array(x);

export _cuda, DeviceArray

In [2]:
#nbx
"""
    griddims = cuda_grid(datadims::Tuple{Vararg{Int}}, 
                         blockdims::Tuple{Vararg{Int}})

Given data dimensions `datadims` and number of threads 
in each dimension `blockdims` returns the respective 
grid dimensions `griddims` such that

    griddims[i] = ceil(Int, datadims[i]/blockdims[i])

"""
function cuda_grid(datadims::Tuple{Vararg{Int}}, blockdims::Tuple{Vararg{Int}})
    griddims = ceil.(Int, datadims./blockdims)
    return griddims
end
export cuda_grid

In [5]:
x  = rand(1000,500,40)
x_ = CuArray(x) 

@btime CuArray($x) samples=3 evals=1;
@btime Array($x_)  samples=3 evals=1;

  20.665 ms (4 allocations: 176 bytes)
  118.607 ms (2 allocations: 152.59 MiB)
